In [30]:
from collections import defaultdict
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  # Para graficar la trayectoria
from ultralytics import YOLO

In [31]:
# Cargar el modelo YOLOv8
model = YOLO("yolov8n.pt")

##Obtener centroides de carros

In [36]:
centroid = []

In [37]:
# Ruta de la imagen
image_path = "C:/Users/areba/OneDrive/Documentos/hackmty2024/yolo_env/videos/chairs.jpg"
image = cv2.imread(image_path)

# Detectar mesas 
results = model(image, classes=[56], conf = 0.5)  

# Extraer las cajas de detección
boxes = results[0].boxes.xyxy.cpu().numpy()  # Coordenadas de las cajas delimitadoras
scores = results[0].boxes.conf.cpu().numpy()  # Confianza de las detecciones

# Dibujar las cajas en la imagen
for box, score in zip(boxes, scores):
    x1, y1, x2, y2 = box
    cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # Caja verde
    cv2.putText(image, f'{score:.2f}', (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    center_x = int(((x2 - x1) // 2) + x1)
    center_y = int(((y2 - y1) // 2 ) + y1)
    centroid.append((center_x, center_y))  


# Mostrar la imagen con las detecciones
cv2.imshow("Detected Tables", image)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 384x640 1 chair, 109.2ms
Speed: 3.0ms preprocess, 109.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


In [83]:
centroid

[(494, 362)]

In [103]:
# Ruta del video
video_path = "C:/Users/areba/OneDrive/Documentos/hackmty2024/yolo_env/videos/prueba4_edit.mp4"
cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = int(cap.get(cv2.CAP_PROP_FPS))


# Diccionario para almacenar el historial completo de las posiciones (x, y) por cada ID
track_history = defaultdict(lambda: [])


In [104]:
# Loop a través de los frames del video
while cap.isOpened():
    success, frame = cap.read()

    if success:
        # Ejecuta el seguimiento con YOLOv8 solo para personas (clase 0)
        results = model.track(frame, persist=True, classes=[0], conf=0.5)

        # Verificar si hay detecciones antes de acceder a las cajas e IDs
        if results[0].boxes is not None:
            boxes = results[0].boxes.xywh.cpu()  # Coordenadas de las cajas delimitadoras

            # Verificar si hay track IDs
            track_ids = results[0].boxes.id
            if track_ids is not None:
                track_ids = track_ids.int().cpu().tolist()  # Convertir a lista si los IDs existen

                # Visualiza los resultados en el frame
                annotated_frame = results[0].plot()

                # Guarda las posiciones de cada ID
                for box, track_id in zip(boxes, track_ids):
                    x, y, w, h = box
                    track = track_history[track_id]
                    track.append((float(x), float(y)))  # Guardar la posición del centro del cuadro (x, y)


                    # Dibujar las líneas de seguimiento
                    points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                    cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)
                    cv2.putText(annotated_frame, "(" + str(float(x)) + "," + str(float(y)) + ")", (int(x), int(y - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                    cv2.circle(annotated_frame, (centroid[0][0], centroid[0][1]), radius=200, color=(0, 255, 0), thickness=2)
                # Mostrar el video con las anotaciones
                cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Salir si se presiona 'q'
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Finaliza cuando el video se haya procesado completamente
        break

# Libera el video y cierra la ventana
cap.release()
cv2.destroyAllWindows()


0: 384x640 (no detections), 188.5ms
Speed: 11.0ms preprocess, 188.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 134.3ms
Speed: 6.2ms preprocess, 134.3ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 120.2ms
Speed: 5.1ms preprocess, 120.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 126.1ms
Speed: 5.0ms preprocess, 126.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 111.5ms
Speed: 2.1ms preprocess, 111.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 97.9ms
Speed: 2.4ms preprocess, 97.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 111.4ms
Speed: 3.5ms preprocess, 111.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 108.3ms
Speed: 2.6ms prepro

In [94]:

# Guardar el historial completo de posiciones en un archivo CSV
track_data = []

# Convertir el diccionario de track_history a una lista de filas
for track_id, positions in track_history.items():
    for frame_num, (x, y) in enumerate(positions):
        track_data.append([track_id, frame_num, x, y])

# Crear un DataFrame para organizar los datos
df = pd.DataFrame(track_data, columns=['ID', 'Frame', 'X', 'Y'])

In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491 entries, 0 to 490
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      491 non-null    int64  
 1   Frame   491 non-null    int64  
 2   X       491 non-null    float64
 3   Y       491 non-null    float64
dtypes: float64(2), int64(2)
memory usage: 15.5 KB


In [98]:
df.shape

(491, 4)

In [99]:
r = 200
c = 0
for i in range(df.shape[0]):
    current = df.iloc[i] 
    x, y = current['X'], current['Y']
    #p1 = np.array([x,y])
    #p2 = np.array([centroid[0][0], centroid[0][1]])
    dist = np.sqrt((x - centroid[0][0])**2 + (y - centroid[0][1])**2)
    print(dist)
    if dist <= r: c+=1
    

1356.06270860406
1355.2658382697055
1352.382841566079
1349.6426071142191
1346.8415824614192
1345.7351093298428
1343.3244407646296
1344.0054364031273
1341.3976218086284
1327.0207464353573
1303.6394222847352
1287.23059048178
1272.5057614918328
1258.3951042030983
1247.573519716072
1239.037374365661
1232.112074059549
1228.362287256367
1227.278454951605
1227.055415568238
1223.6598891104015
1218.1413892744688
1195.0309279650871
1173.9268513806749
1155.4315774785366
1139.3459755960212
1125.0702085935084
1116.9140291364743
1109.5808206603685
1099.8687533539062
1093.7809516386965
1090.0376978321297
1085.317040526496
1076.2711669338262
1062.0262268622569
1045.7618222247359
1031.4963639316502
1018.1765680638373
1006.2022022607274
994.9421853606608
984.7072962636652
973.6241836801416
959.5916168795062
950.9043964953986
931.694089174689
921.8959672943322
904.7012101321451
880.9726459141677
856.5014645001279
830.7211607294951
812.3086446017871
793.1743657686571
778.3013751612654
764.9395498180812
75

In [100]:
c

82

In [ ]:
c